In [1]:
import pandas as pd
from math import comb
from ax.service.ax_client import AxClient
import time
import sys

sys.path.append('../../../0_helper_functions')
import sdlnano as sdl
import calculation as calc

In [2]:
# Design space size calculation


def calculate_combinations(num_variables, total_sum, step):
    steps = int(total_sum / step)
    return comb(steps + num_variables - 1, num_variables - 1)

# Define parameters
step = 0.05
total_sum = 1

# Calculate combinations for each group
count_group1 = calculate_combinations(10, total_sum, step)  # 3 Solid lipids + 3 Polymers + 3 Liquid lipids + 1 Drug
count_group2 = calculate_combinations(4, total_sum, step)   # 3 Surfactants + 1 Water

# Calculate total combinations
total_combinations = count_group1 * count_group2


print("organic design space size: ", round(count_group1/(1e6),2), 'Million')
print("aqueous design space size: ", round(count_group2/(1e3),2), 'Thousand')
print("design space size: ", round(total_combinations/(1e9),2), 'Billion')


organic design space size:  10.02 Million
aqueous design space size:  1.77 Thousand
design space size:  17.74 Billion


# Check the iteration number

In [3]:
iteration = sdl.get_iteration_number()
print("This is the iteration: #", iteration)

This is the iteration: # 8


# Check the drug

In [4]:
drug = 'ACE'
print("This is the drug:", drug)

This is the drug: ACE


# Optimizer initialization (N/A)

# Generate recommendations

In [5]:
time_start = time.time()

previous_optimizer_file_name = '../iteration_' + str(iteration-1) + '/optimizer/optimizer_load_' + str(iteration-1) + '.json'
# generate recomms
ax_client_init = AxClient.load_from_json_file(previous_optimizer_file_name)
unlabeled_trial, ax_client_design = sdl.generate_trials(ax_client=ax_client_init, num_of_trials = 16, drug = drug, bopt=1)

time_end = time.time()

print(round((time_end-time_start)/60), 'min')

/Users/zeqing/opt/anaconda3/envs/sdlnano_plot/lib/python3.11/site-packages/botorch/acquisition/multi_objective/logei.py:418: RuntimeWarning: `cache_root` is only supported for GPyTorchModels that are not MultiTask models and don't produce a TransformedPosterior. Got a model of type <class 'botorch.models.model.ModelList'>. Setting `cache_root = False`.
  NoisyExpectedHypervolumeMixin.__init__(
[INFO 08-07 08:35:56] ax.service.ax_client: Generated new trial 128 with parameters {'Drug': 87, 'SL_1': 74, 'SL_2': 0, 'SL_3': 0, 'LL_1': 0, 'LL_2': 0, 'LL_3': 0, 'P_1': 0, 'P_2': 0, 'P_3': 0, 'S_1': 0, 'S_2': 0, 'S_3': 2, 'Water': 0} using model BoTorch.
/Users/zeqing/opt/anaconda3/envs/sdlnano_plot/lib/python3.11/site-packages/ax/core/data.py:293: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the re

10 min


In [6]:
unlabeled_trial[['S_1', 'S_2', 'S_3', 'Water']]

,S_1,S_2,S_3,Water
0,0,0,2,0
1,0,0,2,21
2,0,0,2,0
3,0,0,0,0
4,0,0,2,0
5,0,3,2,0
6,0,0,3,0
7,0,0,0,0
8,0,0,2,0
9,0,0,0,0


In [7]:
mask = (unlabeled_trial[['S_1', 'S_2', 'S_3', 'Water']] == 0).all(axis=1)

# print the rows that are about to change
print("Rows to be updated (before):")
print(unlabeled_trial.loc[mask])

# set Water to 100 on those rows
unlabeled_trial.loc[mask, 'Water'] = 100

# print the rows after the update
print("\nRows after update:")
print(unlabeled_trial.loc[mask])

Rows to be updated (before):
    trial_index  Drug  SL_1  SL_2  SL_3  LL_1  LL_2  LL_3  P_1  P_2  ...  S_1  \
3           131    89    65     0     0     0     0     0    0    0  ...    0   
7           135    90    75     0     0     0     0     0    0    0  ...    0   
9           137   100    73     0     0     0     0     0    0    0  ...    0   
10          138   100    73     0     0     0     0     0    0    0  ...    0   
13          141    90    75     0     0     0     0     0    0    0  ...    0   

    S_2  S_3  Water  Solu Solu_STD  Size Size_STD   PDI PDI_STD  
3     0    0      0  None     None  None     None  None    None  
7     0    0      0  None     None  None     None  None    None  
9     0    0      0  None     None  None     None  None    None  
10    0    0      0  None     None  None     None  None    None  
13    0    0      0  None     None  None     None  None    None  

[5 rows x 21 columns]

Rows after update:
    trial_index  Drug  SL_1  SL_2  SL_3  LL_1

In [8]:
unlabeled_trial[['S_1', 'S_2', 'S_3', 'Water']]

,S_1,S_2,S_3,Water
0,0,0,2,0
1,0,0,2,21
2,0,0,2,0
3,0,0,0,100
4,0,0,2,0
5,0,3,2,0
6,0,0,3,0
7,0,0,0,100
8,0,0,2,0
9,0,0,0,100


In [9]:
# process the trails into ratios

unlabeled_trial_processed = sdl.process_trails(unlabeled_trial)
unlabeled_trial_processed.describe()

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water
count,16.000000,16.000000,16.000000,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0000,16.000000,16.000000,16.000000
mean,135.500000,0.541562,0.458437,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0625,0.068750,0.461687,0.407062
std,4.760952,0.045221,0.045221,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2500,0.188746,0.459499,0.485961
min,128.000000,0.383000,0.422000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.000000,0.000000,0.000000
25%,131.750000,0.540000,0.431000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.000000,0.000000,0.000000
50%,135.500000,0.540000,0.460000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.000000,0.400000,0.000000
75%,139.250000,0.569000,0.460000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.000000,1.000000,1.000000
max,143.000000,0.578000,0.617000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0000,0.600000,1.000000,1.000000


In [10]:
unlabeled_trial
unlabeled_trial.describe()

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water
count,16.000000,16.000000,16.00000,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.000000,16.000000,16.000000
mean,135.500000,87.750000,73.93750,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.312500,1.437500,32.750000
std,4.760952,9.248423,5.53135,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.873212,1.263263,47.109093
min,128.000000,57.000000,65.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
25%,131.750000,87.000000,73.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
50%,135.500000,87.000000,74.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,2.000000,0.000000
75%,139.250000,90.250000,74.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,2.000000,100.000000
max,143.000000,100.000000,92.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,3.000000,4.000000,100.000000


In [11]:
# Convert ratios to volumes

transfer = calc.converter(unlabeled_trial_processed)
transfer.describe()

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water,Solvent
count,16.000000,16.000000,16.000000,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.000000,16.000000,16.000000,16.0
mean,135.500000,189.546875,160.453125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,62.5,68.750000,461.687500,407.062500,0.0
std,4.760952,15.827313,15.827313,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,250.0,188.745861,459.498526,485.960968,0.0
min,128.000000,134.050000,147.700000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0
25%,131.750000,189.000000,150.850000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0
50%,135.500000,189.000000,161.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,400.000000,0.000000,0.0
75%,139.250000,199.150000,161.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1000.000000,1000.000000,0.0
max,143.000000,202.300000,215.950000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1000.0,600.000000,1000.000000,1000.000000,0.0


In [12]:
#dead_volume = 3.0 # mL

round(transfer[['Drug','SL_1', 'SL_2', 'SL_3', 'LL_1', 'LL_2', 'LL_3', 'P_1', 'P_2', 'P_3', 'S_1', 'S_2', 'S_3', 'Water', 'Solvent']].sum()/1000,1)

Drug       3.0
SL_1       2.6
SL_2       0.0
SL_3       0.0
LL_1       0.0
LL_2       0.0
LL_3       0.0
P_1        0.0
P_2        0.0
P_3        0.0
S_1        1.0
S_2        1.1
S_3        7.4
Water      6.5
Solvent    0.0
dtype: float64

In [13]:
# file path
ax_client_design_path = "optimizer/optimizer_design_" + str(iteration) + ".json"

unlabeled_trial_path = "data/unlabeled_" + str(iteration) + ".xlsx"

unlabeled_trial_processed_path = "data/unlabeled_processed_" + str(iteration) + ".xlsx"

transfer_path = 'data/transfer_' + str(iteration) + '.xlsx'


In [14]:
ax_client_design.save_to_json_file(ax_client_design_path)

unlabeled_trial.to_excel(unlabeled_trial_path, index=False)

unlabeled_trial_processed.to_excel(unlabeled_trial_processed_path, index=False)

transfer.to_excel(transfer_path, index = False)

# Generate new protocol

In [15]:
# Paths
template_protocol = '../../templates/OT_protocol_template.py'

output_script_path_1 = 'protocol/iteration_' + str(iteration) + '_OT_2_protocol_1.py'
output_script_path_2 = 'protocol/iteration_' + str(iteration) + '_OT_2_protocol_2.py'



In [16]:
# Update the script with Excel data
updated_script = calc.update_transfer_script(template_protocol, transfer_path, output_script_path_1, output_script_path_2) 


In [17]:
updated_script

('protocol/iteration_8_OT_2_protocol_1.py',
 'protocol/iteration_8_OT_2_protocol_2.py')

# Perform experiment

# Results

In [18]:
results = calc.parameters_and_complexity(iteration)
results

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water,Complexity,Complexity_STD
0,128,0.540,0.460,0,0,0,0,0,0,0,0,0,0.0,1.000,0.000,2,0
1,129,0.540,0.460,0,0,0,0,0,0,0,0,0,0.0,0.087,0.913,2,0
2,130,0.540,0.460,0,0,0,0,0,0,0,0,0,0.0,1.000,0.000,2,0
3,131,0.578,0.422,0,0,0,0,0,0,0,0,0,0.0,0.000,1.000,1,0
4,132,0.540,0.460,0,0,0,0,0,0,0,0,0,0.0,1.000,0.000,2,0
5,133,0.540,0.460,0,0,0,0,0,0,0,0,0,0.6,0.400,0.000,3,0
6,134,0.569,0.431,0,0,0,0,0,0,0,0,0,0.0,1.000,0.000,2,0
7,135,0.545,0.455,0,0,0,0,0,0,0,0,0,0.0,0.000,1.000,1,0
8,136,0.540,0.460,0,0,0,0,0,0,0,0,0,0.0,1.000,0.000,2,0
9,137,0.578,0.422,0,0,0,0,0,0,0,0,0,0.0,0.000,1.000,1,0


In [19]:
size_raw = calc.size_raw(iteration)
size_raw

,Data Quality,Item,Size,PD Index
0,Good,A1,231.178206,0.168975
1,Good,A2,225.165522,0.211577
2,Good,A3,229.179868,0.215173
3,Good,A4,219.777477,0.369719
4,Caution,A5,227.170947,0.526618
5,Good,A6,214.328524,0.377873
6,Good,B1,228.776087,0.265918
7,Good,B2,228.803951,0.230700
8,Caution,B3,274.907051,0.570796
9,Good,B4,234.968691,0.177040


In [20]:
size_processed = calc.process_formulations(size_raw)
size_processed

,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality
0,Formulation 1,228.507866,3.062153,0.198575,0.025697,1
1,Formulation 2,217.053000,3.852992,0.373796,0.005766,1
2,Formulation 3,228.790019,0.019703,0.248309,0.024903,1
3,Formulation 4,239.962478,7.062281,0.221458,0.062816,1
4,Formulation 5,243.892590,4.975963,0.323892,0.035076,1
5,Formulation 6,172.488412,1.893027,0.257585,0.062059,1
6,Formulation 7,238.978469,2.977501,0.317630,0.011050,1
7,Formulation 8,1000.000000,0.000000,1.000000,0.000000,0
8,Formulation 9,249.524539,5.272500,0.342285,0.026595,1
9,Formulation 10,236.802385,12.341527,0.198066,0.013344,1


In [21]:
results_final = pd.concat([results, size_processed], axis=1)
results_final

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,...,S_3,Water,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality
0,128,0.540,0.460,0,0,0,0,0,0,0,...,1.000,0.000,2,0,Formulation 1,228.507866,3.062153,0.198575,0.025697,1
1,129,0.540,0.460,0,0,0,0,0,0,0,...,0.087,0.913,2,0,Formulation 2,217.053000,3.852992,0.373796,0.005766,1
2,130,0.540,0.460,0,0,0,0,0,0,0,...,1.000,0.000,2,0,Formulation 3,228.790019,0.019703,0.248309,0.024903,1
3,131,0.578,0.422,0,0,0,0,0,0,0,...,0.000,1.000,1,0,Formulation 4,239.962478,7.062281,0.221458,0.062816,1
4,132,0.540,0.460,0,0,0,0,0,0,0,...,1.000,0.000,2,0,Formulation 5,243.892590,4.975963,0.323892,0.035076,1
5,133,0.540,0.460,0,0,0,0,0,0,0,...,0.400,0.000,3,0,Formulation 6,172.488412,1.893027,0.257585,0.062059,1
6,134,0.569,0.431,0,0,0,0,0,0,0,...,1.000,0.000,2,0,Formulation 7,238.978469,2.977501,0.317630,0.011050,1
7,135,0.545,0.455,0,0,0,0,0,0,0,...,0.000,1.000,1,0,Formulation 8,1000.000000,0.000000,1.000000,0.000000,0
8,136,0.540,0.460,0,0,0,0,0,0,0,...,1.000,0.000,2,0,Formulation 9,249.524539,5.272500,0.342285,0.026595,1
9,137,0.578,0.422,0,0,0,0,0,0,0,...,0.000,1.000,1,0,Formulation 10,236.802385,12.341527,0.198066,0.013344,1


In [22]:
# µg/mL
results_final['Solu'] = results_final['Drug'] * 600 / 0.3
results_final['Solu_STD'] = 0

results_final.loc[results_final['Formulation Quality'] == 0, 'Solu'] = 0
results_final

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,...,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality,Solu,Solu_STD
0,128,0.540,0.460,0,0,0,0,0,0,0,...,2,0,Formulation 1,228.507866,3.062153,0.198575,0.025697,1,1080.0,0
1,129,0.540,0.460,0,0,0,0,0,0,0,...,2,0,Formulation 2,217.053000,3.852992,0.373796,0.005766,1,1080.0,0
2,130,0.540,0.460,0,0,0,0,0,0,0,...,2,0,Formulation 3,228.790019,0.019703,0.248309,0.024903,1,1080.0,0
3,131,0.578,0.422,0,0,0,0,0,0,0,...,1,0,Formulation 4,239.962478,7.062281,0.221458,0.062816,1,1156.0,0
4,132,0.540,0.460,0,0,0,0,0,0,0,...,2,0,Formulation 5,243.892590,4.975963,0.323892,0.035076,1,1080.0,0
5,133,0.540,0.460,0,0,0,0,0,0,0,...,3,0,Formulation 6,172.488412,1.893027,0.257585,0.062059,1,1080.0,0
6,134,0.569,0.431,0,0,0,0,0,0,0,...,2,0,Formulation 7,238.978469,2.977501,0.317630,0.011050,1,1138.0,0
7,135,0.545,0.455,0,0,0,0,0,0,0,...,1,0,Formulation 8,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
8,136,0.540,0.460,0,0,0,0,0,0,0,...,2,0,Formulation 9,249.524539,5.272500,0.342285,0.026595,1,1080.0,0
9,137,0.578,0.422,0,0,0,0,0,0,0,...,1,0,Formulation 10,236.802385,12.341527,0.198066,0.013344,1,1156.0,0


In [23]:
results_normalized = sdl.normalize(results_final)
results_normalized[['Formulation Quality','Size', 'Size_STD', 'Solu', 'Solu_STD', 'PDI', 'PDI_STD','Complexity']]

,Formulation Quality,Size,Size_STD,Solu,Solu_STD,PDI,PDI_STD,Complexity
0,1,0.228508,0.003062,0.540,0.0,0.198575,0.025697,0.166667
1,1,0.217053,0.003853,0.540,0.0,0.373796,0.005766,0.166667
2,1,0.228790,0.000020,0.540,0.0,0.248309,0.024903,0.166667
3,1,0.239962,0.007062,0.578,0.0,0.221458,0.062816,0.083333
4,1,0.243893,0.004976,0.540,0.0,0.323892,0.035076,0.166667
5,1,0.172488,0.001893,0.540,0.0,0.257585,0.062059,0.250000
6,1,0.238978,0.002978,0.569,0.0,0.317630,0.011050,0.166667
7,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.083333
8,1,0.249525,0.005272,0.540,0.0,0.342285,0.026595,0.166667
9,1,0.236802,0.012342,0.578,0.0,0.198066,0.013344,0.083333


In [24]:
results_normalized = sdl.normalize(results_final)
results_normalized[['Formulation Quality','Size', 'Size_STD', 'Solu', 'Solu_STD', 'PDI', 'PDI_STD','Complexity']]

,Formulation Quality,Size,Size_STD,Solu,Solu_STD,PDI,PDI_STD,Complexity
0,1,0.228508,0.003062,0.540,0.0,0.198575,0.025697,0.166667
1,1,0.217053,0.003853,0.540,0.0,0.373796,0.005766,0.166667
2,1,0.228790,0.000020,0.540,0.0,0.248309,0.024903,0.166667
3,1,0.239962,0.007062,0.578,0.0,0.221458,0.062816,0.083333
4,1,0.243893,0.004976,0.540,0.0,0.323892,0.035076,0.166667
5,1,0.172488,0.001893,0.540,0.0,0.257585,0.062059,0.250000
6,1,0.238978,0.002978,0.569,0.0,0.317630,0.011050,0.166667
7,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.083333
8,1,0.249525,0.005272,0.540,0.0,0.342285,0.026595,0.166667
9,1,0.236802,0.012342,0.578,0.0,0.198066,0.013344,0.083333


In [25]:
labeled_data_path = 'data/labeled_' + str(iteration) + '.xlsx'
labeled_norm_path = 'data/labeled_norm_' + str(iteration) + '.xlsx'


In [26]:
results_final.to_excel(labeled_data_path, index=False)
results_normalized.to_excel(labeled_norm_path, index=False)

# Load the labeled data to update the optimizer

In [27]:
labeled_norm = pd.read_excel(labeled_norm_path)
labeled_norm.head()

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,...,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality,Solu,Solu_STD
0,128,0.540,0.460,0,0,0,0,0,0,0,...,0.166667,0,Formulation 1,0.228508,0.003062,0.198575,0.025697,1,0.540,0
1,129,0.540,0.460,0,0,0,0,0,0,0,...,0.166667,0,Formulation 2,0.217053,0.003853,0.373796,0.005766,1,0.540,0
2,130,0.540,0.460,0,0,0,0,0,0,0,...,0.166667,0,Formulation 3,0.228790,0.000020,0.248309,0.024903,1,0.540,0
3,131,0.578,0.422,0,0,0,0,0,0,0,...,0.083333,0,Formulation 4,0.239962,0.007062,0.221458,0.062816,1,0.578,0
4,132,0.540,0.460,0,0,0,0,0,0,0,...,0.166667,0,Formulation 5,0.243893,0.004976,0.323892,0.035076,1,0.540,0


In [28]:
ax_client_design = AxClient.load_from_json_file(ax_client_design_path)


In [29]:
loaded_optimizer_path = "optimizer/optimizer_load_" + str(iteration) + ".json"

In [30]:
loaded_ax_client = sdl.load_labeled_data(ax_client_design, labeled_norm_path)
loaded_ax_client.save_to_json_file(loaded_optimizer_path)

[INFO 08-07 10:43:49] ax.service.ax_client: Completed trial 128 with data: {'Solu': (0.54, 0), 'Size': (0.228508, 0.003062), 'PDI': (0.198575, 0.025697)}.
[INFO 08-07 10:43:49] ax.service.ax_client: Completed trial 129 with data: {'Solu': (0.54, 0), 'Size': (0.217053, 0.003853), 'PDI': (0.373796, 0.005766)}.
[INFO 08-07 10:43:49] ax.service.ax_client: Completed trial 130 with data: {'Solu': (0.54, 0), 'Size': (0.22879, 2e-05), 'PDI': (0.248309, 0.024903)}.
[INFO 08-07 10:43:49] ax.service.ax_client: Completed trial 131 with data: {'Solu': (0.578, 0), 'Size': (0.239962, 0.007062), 'PDI': (0.221458, 0.062816)}.
[INFO 08-07 10:43:49] ax.service.ax_client: Completed trial 132 with data: {'Solu': (0.54, 0), 'Size': (0.243893, 0.004976), 'PDI': (0.323892, 0.035076)}.
[INFO 08-07 10:43:49] ax.service.ax_client: Completed trial 133 with data: {'Solu': (0.54, 0), 'Size': (0.172488, 0.001893), 'PDI': (0.257585, 0.062059)}.
[INFO 08-07 10:43:49] ax.service.ax_client: Completed trial 134 with data